In [1]:
import tensorflow as tf
import keras as kr
from keras import layers
from keras import regularizers
import keras_tuner

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

2023-01-08 00:36:14.491226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 00:36:14.600737: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-08 00:36:14.604715: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 00:36:14.604730: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# Load the dataset

In [10]:
df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/MetaGenAutoencoder/Data/KO_metaG.norm.txt",sep="\t")
df = df.iloc[:,2-len(df.columns):]
print(df.shape)
df.head(5)

train_data = np.array(df)
print(train_data.shape)
inputlen = len(df.columns)
print(inputlen)

(180, 9024)
(180, 9024)
9024


# Define model and hyperparameters to be tuned

In [11]:
def autoencoder(units, lr):
    inputlen = 9024
    encoding_dim = 3
    # This is my model
    model = kr.Sequential()
    model.add(kr.Input(shape=(inputlen,)))
    model.add(layers.Dense(units=units*2, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(units=units, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(encoding_dim, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(units=units, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(units=units*2, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(inputlen, activation='linear'))
    
   # This model maps an input to its reconstruction
    autoencoder = kr.Model(model.input, model.output)
    opt = kr.optimizers.Adam(learning_rate=lr)
    autoencoder.compile(optimizer=opt, loss='mse',metrics=["mean_squared_error"])
    return autoencoder

def build_model(hp):
    units = hp.Int("units", min_value=100, max_value=1000, sampling="log")
    lr = hp.Float("lr",min_value=1e-4, max_value=1e-2, sampling="log")
    model = autoencoder(units,lr)
    return model

build_model( keras_tuner.HyperParameters())

# Search the best model

In [12]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="mean_squared_error",
    max_trials=10,
    executions_per_trial=3,
    seed = 333,
    overwrite=True,
    directory="/home/vmh/vmhdocs/Research/Inria/Anl/MetaGenAutoencoder/Code/autoenc001",
    project_name="tuner",
)


tuner.search_space_summary()
tb = kr.callbacks.TensorBoard(log_dir="./tuner_tb", write_images=True,update_freq="epoch")
tuner.search(train_data, train_data, epochs=20,callbacks=[tb])

Trial 10 Complete [00h 01m 42s]
mean_squared_error: 0.0077662270826598006

Best mean_squared_error So Far: 0.003190848510712385
Total elapsed time: 00h 06m 35s
INFO:tensorflow:Oracle triggered exit


# Save the best model

In [13]:
# Get best model.
best_model = tuner.get_best_models(num_models=1)
best_model = best_model[0]

# Build the model.
best_model.build(input_shape=(inputlen,))
best_model.summary()

best_hps = tuner.get_best_hyperparameters(1)
best_hps = best_hps[0]
# Build the model with the best hp.
autoencoder = build_model(best_hps)
autoencoder.save('./autoencoder_metagendata.h5')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9024)]            0         
                                                                 
 dense (Dense)               (None, 1180)              10649500  
                                                                 
 batch_normalization (BatchN  (None, 1180)             4720      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 590)               696790    
                                                                 
 batch_normalization_1 (Batc  (None, 590)              2360      
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 3)                 1773  